In [1]:
import pandas as pd
import numpy as np
from word import Word
%load_ext autoreload
%autoreload 2

In [3]:
datatype = {'round': str, 
            'session': str, 
            'participant': str,
            'line_part': str,
            'stimuli_image': str,
            'stimuli_text': str,
            'duration': 'int64',
            'x_cord': 'float64',
            'y_cord': 'float64',
            'aoi_x': 'float64',
            'aoi_y': 'float64', 
            'aoi_width': 'float64', 
            'aoi_height': 'float64',
            'token': str,
            'length': 'int64'}

df = df.astype(datatype)

In [12]:
df.to_csv('freq_result.csv')

In [44]:
from word import Word
from tqdm import tqdm

In [72]:
lexicon = {}

participants = {}

for row in tqdm(df.iterrows()):
    
    index = row[0]
    cols = row[1]
    
    round_id = index[0]
    session_id = index[1]
    file_id = round_id + session_id
    
    participant_id = index[2:5]
    
    line_part = cols['line_part']
    _, line_id, _, part_id = line_part.split()
    
    timestamp = cols['timestamp']
    duration = cols['duration']
    length = cols['length']
    frequency = cols['CobMln']
    predict = 0
    level = 0
    
    token = cols['token']
    
    key = f'{file_id}_{line_id}_{part_id}_{participant_id}'
    
    if lexicon.get(key, 0) == 0:
        lexicon[key] = Word(file_id, line_part, token)
        
    lexicon[key].add(participant=participant_id, 
                     duration=duration, 
                     start_time=timestamp, 
                     length=length, 
                     freq=frequency, 
                     predict=predict,
                     level=level)
    
    if participants.get(file_id, 0) == 0:
        participants[file_id] = []
    
    if participant_id not in participants[file_id]:
        participants[file_id].append(participant_id)

537666it [00:42, 12728.76it/s]


In [57]:
import json
j = json.dumps(participants)
with open('participants.txt', 'w') as file:
    file.write(j)

In [73]:
headers = ['file',
           'line',
           'part',
           'token',
           'subject',
           'SFD',
           'FFD',
           'GD',
           'TT',
           'PrF',
           'Pr1',
           'Pr2',
           'PrS',
           'freq',
           'length']

result = pd.DataFrame(columns=headers)

In [76]:
for key, word in tqdm(lexicon.items()):
    file, line, part, subject = key.split('_')
    token = word.get_text()
    SFD = word.single_fix_duration()
    FFD = word.first_fix_duration()
    GD = word.gaze_duration()
    TT = word.total_time()
    PrF = word.fix_probability()
    Pr1 = word.prob_of_one_fix()
    Pr2 = word.prob_of_2_or_more_fix()
    PrS = word.prob_of_skip()
    freq = word.get_freq()
    length = word.get_leng()
    
    values = [file, line, part, token, subject, SFD, FFD, GD, TT, PrF, Pr1, Pr2, PrS, freq, length]
    
    dic = dict(zip(headers, values))
    
    result = result.append(dic, ignore_index=True)

100%|█████████████████████████████████| 230091/230091 [1:12:25<00:00, 52.95it/s]


In [79]:
result.to_csv('Eight_Metrics.csv', index=False)

# Add normalize metrics
# Inspect length

In [2]:
import pandas as pd

df = pd.read_csv('Eight_Metrics.csv')

df = df.dropna(subset=['token'])

df['nSFD'] = df.loc[:,'SFD'] / df['length']
df['nFFD'] = df.loc[:,'FFD'] / df['length']
df['nGD'] = df.loc[:,'GD'] / df['length']
df['nTT'] = df.loc[:,'TT'] / df['length']

df

,file,line,part,token,subject,SFD,FFD,GD,TT,PrF,Pr1,Pr2,PrS,freq,length,nSFD,nFFD,nGD,nTT
0,11,12,3,brought,1,NaN,120.0,120.0,836.0,0.00,0.00,0.00,1.00,50,7,NaN,17.142857,17.142857,119.428571
1,11,13,3,broker,1,NaN,120.0,120.0,700.0,0.00,0.00,0.00,1.00,3,6,NaN,20.000000,20.000000,116.666667
2,11,12,4,to,1,NaN,161.0,161.0,353.0,0.00,0.00,0.00,1.00,19942,2,NaN,80.500000,80.500000,176.500000
3,11,13,4,to,1,161.0,161.0,161.0,161.0,0.00,0.00,0.00,1.00,19942,2,80.50,80.500000,80.500000,80.500000
4,11,12,5,arrange,1,NaN,74.0,74.0,695.0,0.00,0.00,0.00,1.00,7,7,NaN,10.571429,10.571429,99.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230685,92,24,8,of,314,NaN,267.0,267.0,732.0,0.07,0.00,0.07,0.93,30106,2,NaN,133.500000,133.500000,366.000000
230686,92,25,5,of,314,NaN,267.0,267.0,829.0,0.07,0.00,0.07,0.93,30106,2,NaN,133.500000,133.500000,414.500000
230687,92,25,1,in,314,NaN,136.0,136.0,363.0,0.07,0.00,0.07,0.93,18464,2,NaN,68.000000,68.000000,181.500000
230688,92,3,3,if,314,NaN,181.0,181.0,363.0,0.07,0.00,0.07,0.93,3006,2,NaN,90.500000,90.500000,181.500000
